In [1]:
import json
import numpy as np
from sklearn.preprocessing import OneHotEncoder
with open('var.json') as f:
    var_temp = json.load(f)

In [29]:
dfte = pd.read_csv('ukb46359.csv')

/opt/anaconda3/envs/bigdatalab_cpu_202101/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3156: DtypeWarning: Columns (17,191,1298,1300,2843,2844,2845,2846,2847,2848,2849,2850,2851,2852,2853,2854,2855,2856,2857,2858,2859,2860,2861,2862,2863,2864,2865,2866,2867,2868,2869,2870,2871,2872,2873,2874,2875,2876,2877,2878,2879,2880,2881,2882,2883,2884,2885,2886,2887,2888,2889,2890,2891,2892,2893,2894,2895,2896,2897,2898,2899,2900,2901,2902,2903,2904,2905,2906,2907,2908,2909,2910,2911,2912,2913,2914,2915,2916,2917,2918,2919,2924,2969,2970,2971,2972,2973,2974,2975,2976,2977,2978,2979,2980,2981,2982,2983,2984,2985,2986,2987,2988,2989,2990,2991,2992,2993,2994,2995,2996,2997,2998,2999,3000,3001,3002,3003,3004,3005,3006,3007,3008,3009,3010,3011,3012,3013,3014,3015,3016,3017,3018,3019,3020,3021,3022,3023,3024,3025,3026,3027,3028,3029,3030,3031,3032,3033,3034,3035,3036,3037,3038,3039,3040,3041,3042,3043,3044,3045,3046,3047,3048,3049,3050,3051,3052,3053,3054,3055,3056,3057,3058,3059,3060,3

In [30]:
print(dfte.shape)

(502481, 4009)


In [2]:
# Adding to the JSON the ICD codes fields
icd10 = []
icd9 = []

for i in range(75):
    var_temp["41202-0."+ str(i)] = "ICD10_"+str(i)
    icd10.append("ICD10_"+str(i))

for i in range(28):
    var_temp["41203-0."+ str(i)] = "ICD9_"+str(i)
    icd9.append("ICD9_"+str(i))

In [3]:
import pandas as pd

df = pd.read_csv('ukb46359.csv', usecols = var_temp.keys()) # just the columns to select, the ones in the json
df = df.rename(columns = var_temp) # rename the columns with the field of the json
df.shape

/opt/anaconda3/envs/bigdatalab_cpu_202101/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3156: DtypeWarning: Columns (2843,2844,2845,2846,2847,2848,2849,2850,2851,2852,2853,2854,2855,2856,2857,2858,2859,2860,2861,2862,2863,2864,2865,2866,2867,2868,2869,2870,2871,2872,2873,2874,2875,2876,2877,2878,2879,2880,2881,2882,2883,2884,2885,2886,2887,2888,2889,2890,2891,2892,2893,2894,2895,2896,2897,2898,2899,2900,2901,2902,2903,2904,2905,2906,2907,2908,2909,2910,2911,2912,2913,2914,2915,2916,2917,2918,2919,2924) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(502481, 328)

In [4]:
#Add the new dataset containing age and merge it with the previous one
df_add = pd.read_csv('ukb49112.csv', usecols = ['eid', '34-0.0','189-0.0','1767-0.0','1767-1.0','1767-2.0','1767-3.0'])
df_add['adoptedasachild'] = df_add[['1767-0.0','1767-1.0','1767-2.0','1767-3.0']].replace({-3:None,-1:None}).max(axis=1)
df_add = df_add.drop(['1767-0.0','1767-1.0','1767-2.0','1767-3.0'], axis = 1)

df_add = df_add.rename(columns = {'eid' : 'id', '34-0.0': 'age', '189-0.0':'townsenddeprivation'})

df = df.merge(df_add, how='left', on='id')
df.shape

(502481, 331)

In [5]:
df_ = df.copy()

#sum the MET values to obtain a single column
MET_cols = ["METminutesperweekforwalking", "METminutesperweekformoderateactivity", "METminutesperweekforvigorousactivity"]
df_['summedMETminutes']= df_[MET_cols].sum(axis=1)

#replace -7: None of the above with 7 and -3: Prefer not to answer with nan in qualification columns
#7 as qualification will represent another category
for a in range(6):
    df_["qualifications."+str(a)].replace({-7: 7, -3: 'nan'}, inplace=True)
#Add values in qualification columns to obtain a single column

qual_cols = ["qualifications.0","qualifications.1","qualifications.2","qualifications.3","qualifications.4","qualifications.5"]
df_['qualification']= df_[qual_cols].sum(axis=1)   
    
df_.shape

(502481, 333)

In [6]:
# IC_code_cvds --> look the ones that I'm interested

# ICD10 Diseases Of The Circulatory System: I00-I99
ic10_code_cvd_ = ['I0' + str(i) for i in range(10)]
ic10_code_cvd_.extend(['I' + str(i) for i in range(10,100)])
#remove codes related to hypertension
ic10_code_cvd = [e for e in ic10_code_cvd_ if e not in ('I10', 'I11','I12', 'I13', 'I14', 'I15', 'I16')]
# ICD9 Diseases Of The Circulatory System: 390-459
ic9_code_cvd_ = list(range(390,460))
#remove codes related to hypertension
ic9_code_cvd = [e for e in ic9_code_cvd_ if e not in (401, 402, 403, 404, 405, 406)]

df_['cvd_disease_from_icd'] = 0

value = ''*len(df_) 
value_cvd = []

#check for icd10 diagnosis
for col in icd10:
    value += df_[col].map(str)

value = list(value)

for l in range(len(value)):
    #check for CVD
    if any(code in value[l] for code in ic10_code_cvd):
        
        value_cvd.append(1)
    else: 
        value_cvd.append(0)

df_['cvd_disease_from_icd'] = value_cvd

#check for icd9 diagnosis

#find a rows which contain any of icd9 diagnosis code for cvd
df1 = df_.loc[np.where(df_.loc[:,icd9].isin(ic9_code_cvd)==True)[0]]
#add 1 to 'cvd_disease_from_icd' to the corresponding row
df_.loc[df1.index, 'cvd_disease_from_icd'] = 1


In [7]:
# IC_code_depression

#ICD10 code list for Mood [affective] disorders: F30-F39
ic10_code_depr = ['F' + str(i) for i in range(30,40)]
#ic10_code_depr.extend(['F' + str(i) for i in range(10,100)])

#ICD9 code list for Mental Disorders: 290-319
ic9_code_depr = list(range(290,320))

df_['depr_disease_from_icd'] = 0

value = ''*len(df_) 
value_depr = []

for col in icd10:
    value += df_[col].map(str)

value = list(value)

for l in range(len(value)):
    #check for Depression/mental disorders
    if any(code in value[l] for code in ic10_code_depr):
        
        value_depr.append(1)
    else: 
        value_depr.append(0)
        
df_['depr_disease_from_icd'] = value_depr

#check for icd9 diagnosis

#find a rows which contain any of icd9 diagnosis code for depression
df1 = df_.loc[np.where(df_.loc[:,icd9].isin(ic9_code_depr)==True)[0]]
#add 1 to 'cvd_disease_from_icd' to the corresponding row
df_.loc[df1.index, 'depr_disease_from_icd'] = 1

In [8]:
# IC_code_diabetes

#ICD10 code list for Diabetes mellitus: E08-E13
ic10_code_diab = ['E08', 'E09', 'E10', 'E11', 'E13']

#ICD9 code list for Diabetes mellitus: 250
ic9_code_diab = [250]

df_['diab_disease_from_icd'] = 0

value = ''*len(df_) 
value_diab = []

for col in icd10:
    value += df_[col].map(str)

value = list(value)

for l in range(len(value)):
    #check for Depression/mental disorders
    if any(code in value[l] for code in ic10_code_diab):
        
        value_diab.append(1)
    else: 
        value_diab.append(0)
        
df_['diab_disease_from_icd'] = value_diab

#check for icd9 diagnosis

#find a rows which contain any of icd9 diagnosis code for diabetes
df1 = df_.loc[np.where(df_.loc[:,icd9].isin(ic9_code_diab)==True)[0]]
#add 1 to 'cvd_disease_from_icd' to the corresponding row
df_.loc[df1.index, 'diab_disease_from_icd'] = 1

In [9]:
#include other columns for diagnosis of depression, which include:
#    "2090-0.0":"seendoctordepranxietytime0", values: [ 1.  0. -1. -3. nan]
#    "2090-1.0":"seendoctordepranxietytime1"
#    "2090-2.0":"seendoctordepranxietytime2"
#    "2090-3.0":"seendoctordepranxietytime3"
#    "2100-0.0":"seenpsychidepranxietytime0", values: [ 1.  0. -3. -1. nan]
#    "2100-1.0":"seenpsychidepranxietytime1", 
#    "2100-2.0":"seenpsychidepranxietytime2",
#    "2100-3.0":"seenpsychidepranxietytime3",
#    "20126-0.0":"Bipolarandmajordepressionstatus", values: [ 3. nan  0.  4.  5.  2.  1.]
#    "20448-0.0":"Professionalinformedaboutdepression", values: [   1.   nan    0. -818. -121.]
#    "20546-0.1": "medicinedepression1", values: [   1.,   nan,    3.,    4., -818.]
#    "20546-0.2": "medicinedepression2",
#    "20546-0.3": "medicinedepression3",
#add a new column for depression
df_['depression'] = 0
# add 1 to the column depression if the below conditions meet
df_.loc[(df_['seendoctordepranxietytime0'] == 1) | (df_['seendoctordepranxietytime1'] == 1) | (df_['seendoctordepranxietytime2'] == 1) | (df_['seendoctordepranxietytime3'] == 1) | (df_['seenpsychidepranxietytime0'] == 1) | (df_['seenpsychidepranxietytime1'] == 1) | (df_['seenpsychidepranxietytime2'] == 1) | (df_['seenpsychidepranxietytime3'] == 1) | (df_['Professionalinformedaboutdepression'] == 1) | (df_['Bipolarandmajordepressionstatus'].isin([1,2,3,4,5])),'depression'] = 1
#additional columns related to diagnosis of depression 
df_.loc[(df_['medicinedepression1'].isin([1,3,4])) | (df_['medicinedepression2'].isin([1,3,4])) | (df_['medicinedepression3'].isin([1,3,4])) , 'depression'] = 1

#As a final step, add 1 to column 'depression' where 'depr_disease_from_icd' = 1
df_.loc[(df_['depr_disease_from_icd'] == 1), 'depression'] = 1

In [10]:
#include other columns for diagnosis of diabetes, which include:
#    "2443-1.0":"diabetesdiagnosistime0", values; [ 0.  1. -1. -3. nan]
#    "2443-2.0":"diabetesdiagnosistime1", values; [ 0.  1. -1. -3. nan]
#    "2443-3.0":"diabetesdiagnosistime2", values; [ 0.  1. -1. -3. nan]
#    "2443-4.0":"diabetesdiagnosistime3", values; [ 0.  1. -1. -3. nan]
#    "2986-0.0":"Startedinsulinwithinoneyeardiagnosisofdiabetes", values: [nan  0.  1. -1. -3.]

#add a new column for diabetes
df_['diabetes'] = 0

df_.loc[(df_["diabetesdiagnosistime0"] == 1) |(df_["diabetesdiagnosistime1"] == 1) | (df_["diabetesdiagnosistime2"] == 1) | (df_["diabetesdiagnosistime3"] == 1) | (df_['Startedinsulinwithinoneyeardiagnosisofdiabetes'] == 1), 'diabetes'] = 1

#As a final step, add 1 to column 'diabetes' where 'diab_disease_from_icd' = 1
df_.loc[(df_['diab_disease_from_icd'] == 1), 'diabetes'] = 1

In [11]:
#include other columns for diagnosis of cardiovascular disease, which include:
#    "6150-0.0": "cvd0.time0", values: [-7.  1.  4.  2. -3.  3. nan]
#    "6150-0.0": "cvd0.time0",
#    "6150-0.1": "cvd1.time0",
#    "6150-0.2": "cvd2.time0",
#    "6150-0.3": "cvd3.time0",
#    "6150-1.0": "cvd0.time1",
#    "6150-1.1": "cvd1.time1",
#    "6150-1.2": "cvd2.time1",
#    "6150-1.3": "cvd3.time1",
#    "6150-2.0": "cvd0.time2",
#    "6150-2.1": "cvd1.time2",
#    "6150-2.2": "cvd2.time2",
#    "6150-2.3": "cvd3.time2",

#add a new column for diabetes
df_['cvd'] = 0

cvd_cols = ["cvd0.time0", "cvd1.time0","cvd2.time0", "cvd3.time0","cvd0.time1","cvd1.time1", "cvd2.time1", 
            "cvd3.time1", "cvd0.time2", "cvd1.time2", "cvd2.time2", "cvd3.time2"]

df1 = df_.loc[np.where(df_.loc[:,cvd_cols].isin([1,2,3,4])==True)[0]]
df_.loc[df1.index, 'cvd'] = 1

#Add 1 to column 'cvd' where 'cvd_disease_from_icd' = 1
df_.loc[(df_['cvd_disease_from_icd'] == 1), 'cvd'] = 1

In [12]:
###############################################################################################
# ADD A COLUMN FOT CVDS FROM QUESTIONNAIRE
###############################################################################################

from_other = ["disease" + str(i) + ".time0" for i in range(30)] # columns to check
#these are codes of the disease that were self reported by the subjects
cvd_from_other = ['1066','1067','1068','1074','1075','1076','1077','1078','1079','1080','1081','1082','1083','1086','1087','1088','1093','1094']

#find a rows which contain any of cvd_from_other code in any of from_other columns
df1 = df_.loc[np.where(df_.loc[:,from_other].isin(cvd_from_other)==True)[0]]
#add 1 to 'cvd' in the corresponding row
df_.loc[df1.index, 'cvd'] = 1


In [13]:
# codes for diabetes

diab_from_other = ['1220','1222','1223']

#find a rows which contain any of diab_from_other code in any of from_other columns
df1 = df_.loc[np.where(df_.loc[:,from_other].isin(diab_from_other)==True)[0]]
#add 1 to 'cvd' in the corresponding row
df_.loc[df1.index, 'diabetes'] = 1

In [14]:
# codes of depression
depr_from_other = ['1286','1287']

#find a rows which contain any of depr_from_other code in any of from_other columns
df1 = df_.loc[np.where(df_.loc[:,from_other].isin(depr_from_other)==True)[0]]
#add 1 to 'cvd' in the corresponding row
df_.loc[df1.index, 'depression'] = 1

In [15]:
df_

,id,sex,waistcircum,hipcircum,agecompleted,sleepduration,napduringday,sleeplessness,daytimedozing,currenttobacco,...,townsenddeprivation,adoptedasachild,summedMETminutes,qualification,cvd_disease_from_icd,depr_disease_from_icd,diab_disease_from_icd,depression,diabetes,cvd
0,1000010,0.0,74.0,102.0,NaN,7.0,2.0,3.0,0.0,0.0,...,0.994956,0.0,1235.0,11.0,0,0,0,1,0,1
1,1000028,1.0,120.0,113.0,16.0,9.0,2.0,2.0,0.0,0.0,...,-1.541060,0.0,0.0,0.0,1,0,0,0,0,1
2,1000034,0.0,66.0,88.0,16.0,5.0,1.0,3.0,0.0,0.0,...,-2.182290,0.0,11697.0,0.0,0,0,0,0,0,0
3,1000045,1.0,110.0,117.0,18.0,7.0,2.0,1.0,0.0,1.0,...,-0.821087,0.0,2346.0,0.0,1,0,0,0,0,1
4,1000052,1.0,94.0,100.0,NaN,6.0,2.0,3.0,0.0,0.0,...,-3.782560,0.0,2132.0,11.0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502476,6024770,1.0,110.0,113.0,16.0,7.0,2.0,2.0,0.0,2.0,...,-3.093120,0.0,0.0,0.0,1,0,0,0,0,1
502477,6024784,0.0,102.0,110.0,16.0,6.0,2.0,3.0,0.0,0.0,...,-4.248680,0.0,4146.0,6.0,1,0,0,1,0,1
502478,6024795,1.0,99.0,98.0,-2.0,7.0,2.0,2.0,1.0,0.0,...,1.238560,0.0,0.0,0.0,1,0,1,1,1,1
502479,6024804,0.0,86.0,111.0,NaN,6.0,1.0,3.0,0.0,0.0,...,-2.729560,0.0,1428.0,0.0,0,0,0,0,0,0


In [16]:
#print(df_['depr_disease_from_icd'].value_counts())
print(df_['depression'].value_counts())
#print(df_['diab_disease_from_icd'].value_counts())
print(df_['diabetes'].value_counts())
#print(df_['cvd_disease_from_icd'].value_counts())
print(df_['cvd'].value_counts())

0    304958
1    197523
Name: depression, dtype: int64
0    473480
1     29001
Name: diabetes, dtype: int64
0    286121
1    216360
Name: cvd, dtype: int64


In [17]:
df_c = df_.copy()

In [18]:
# Remove rows with cvd missing data and diabetes diagnosis
check = ["disease" + str(i) + ".time0" for i in range(30)]
check.extend(["cvd0.time0","cvd1.time0","cvd2.time0","cvd3.time0"])
df_c = df_c.dropna(how = 'all', subset = check)

check = ["disease" + str(i) + ".time0" for i in range(30)]
check.extend(["diabetesdiagnosistime0"])
df_c = df_c.dropna(how = 'all', subset = check)

len(df_c)

501948

In [19]:
print(df_c['depression'].value_counts())
print(df_c['diabetes'].value_counts())
print(df_c['cvd'].value_counts())

0    304449
1    197499
Name: depression, dtype: int64
0    472955
1     28993
Name: diabetes, dtype: int64
0    285718
1    216230
Name: cvd, dtype: int64


In [20]:
#Now that columns for 3 target diseases are ready,drop the unnecessary columns from the dataframe
cols = ["seendoctordepranxietytime0",
        "seendoctordepranxietytime1",
        "seendoctordepranxietytime2",
        "seendoctordepranxietytime3",
        "seenpsychidepranxietytime0",
        "seenpsychidepranxietytime1", 
        "seenpsychidepranxietytime2",
        "seenpsychidepranxietytime3",
        "Bipolarandmajordepressionstatus",
        "Professionalinformedaboutdepression", 
        "medicinedepression1", 
        "medicinedepression2",
        "medicinedepression3", 
        "diabetesdiagnosistime0",
        "diabetesdiagnosistime1",
        "diabetesdiagnosistime2",
        "diabetesdiagnosistime3",
        "Startedinsulinwithinoneyeardiagnosisofdiabetes", 
        "cvd_disease_from_icd", 
        "depr_disease_from_icd", 
        "diab_disease_from_icd"]

#getting column names related to questionnaire 
# Code for this feature in UKBB startes with 20002, so these column names can be filtered
col_others = [key[1] for key in var_temp.items() if '20002' in key[0]]

col_to_drop = icd9 + icd10 + cvd_cols + MET_cols + qual_cols + cols + col_others
df_c = df_c.drop(col_to_drop, axis = 1)
df_c.shape

(501948, 95)

In [21]:
for col in df_c.columns:  
    print(col)    

id
sex
waistcircum
hipcircum
agecompleted
sleepduration
napduringday
sleeplessness
daytimedozing
currenttobacco
pasttobacco
cookedvegetable
saladintake
fruitintake
driedfruit
oilyfishintake
nonoilyfish
processedmeat
poultryintake
beefintake
lambintake
porkintake
cheeseintake
milkused
spreadtype
breadintake
breadtype
cerealintake
cerealtype
salttofood
teaintake
coffetype
waterintake
majordietarychang
variationdiet
alcoholintake
breastfedbaby
adoptedaschild
maternalsmoking
freqdepressed
frequnenthusiasm2weeks
freqtensenness2weeks
freqtiredness2weeks
nonbutterspread
diastolicbloodpress
systolicbloodpress
everdepressed
perioddepress
numberdepressper
everdesinteresedweek
currentemployment
smoking
alcoholdrinkerstatus
neuroticismscore
amountalcoholdrunk
everhadknownpersonalcoh
everbeeninjuredrinkalcohol
frequdrinkingalcohol
evertakencannabis
felthatedfamiliychild
physicallyabusedfamiliychild
feltlovedchild
sexuallymolestedchild
someonetakedoctorchild
avoidedactsstresspasthmonth
repeatedistur

In [22]:
#Replace -3: Prefer not to answer and -1: do not know, with nan values
#Replace -7 which represents 'None of the above' in qualification column. 8 can represent a new category
#Replace -10 which represents 'less than 1' for integer columns with 1
#Replace -818 which represents 'Prefer not to answer' with None
#Replace -2 which represents 'never went to school' with 1 (just a small number)
for col in df_c.columns:  
    df_c[col].replace({-3: np.nan, -1: np.nan, -2:1, -7:8, -10:1, -818: np.nan}, inplace=True)
    #print(str(col), df_c[col].isna().sum())

In [23]:
df_c.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 501948 entries, 0 to 502480
Data columns (total 95 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   id                              501948 non-null  int64  
 1   sex                             501948 non-null  float64
 2   waistcircum                     500161 non-null  float64
 3   hipcircum                       500102 non-null  float64
 4   agecompleted                    330607 non-null  float64
 5   sleepduration                   498235 non-null  float64
 6   napduringday                    500559 non-null  float64
 7   sleeplessness                   500947 non-null  float64
 8   daytimedozing                   498718 non-null  float64
 9   currenttobacco                  501131 non-null  float64
 10  pasttobacco                     460333 non-null  float64
 11  cookedvegetable                 494861 non-null  float64
 12  saladintake     

In [24]:
els1 =  ['amountalcoholdrunk',
        'everhadknownpersonalcoh',
        'everbeeninjuredrinkalcohol',
        'frequdrinkingalcohol',
        'evertakencannabis',
        'felthatedfamiliychild',
        'physicallyabusedfamiliychild',
        'feltlovedchild',
        'sexuallymolestedchild',
        'someonetakedoctorchild',
        'avoidedactsstresspasthmonth',
        'repeatedisturbingpastmonth',
        'feltupsetstressfulpastmonth',
        'belittlementbypartneradult',
        'beenconfidingrelationadult',
        'physicalviolenceadult',
        'sexualinterferenceadult',
        'abletopaymortgageadult',
        'beenseriousaccidentlifethrea',
        'diagnosedlifethreateningillnes',
        'victimphysicallyviolentcrime',
        'witnessedsuddenviolentdeath',
        'victimsexualassault']


print("len of df_c: ", len(df_c.columns))
print("len of els1: ", len(els1))
cols = df_c.columns.tolist()

for col in els1:
        cols.remove(col)
#these two dataframes will be our 2 separate datasets
df_ca = df_c[els1]
df_cb = df_c[cols]

print("df_ca shape: ", df_ca.shape)
print("df_cb shape: ", df_cb.shape)

len of df_c:  95
len of els1:  23
df_ca shape:  (501948, 23)
df_cb shape:  (501948, 72)


In [25]:
#copy target disease columns and merge them with df_small dataset
df_small = df_ca.dropna(how = 'all')
print("df_small shape: ", df_small.shape)
targets = df_c.loc[df_small.index]
targets = targets[['diabetes', 'depression', 'cvd']].copy()
df_small = pd.concat([df_small, targets], axis=1).reindex(df_small.index)
print("df_small shape: ", df_small.shape)

df_small shape:  (157287, 23)
df_small shape:  (157287, 26)


In [26]:
#Remove any row or column in the df_cb that is completely nan
df_big = df_cb.dropna(how = 'all')
print("df_big shape: ", df_big.shape)

df_big shape:  (501948, 72)


In [27]:
df_temp = df_big.copy()

els2 = ['hardphysicalactivity',
        'moderatephysicalactivity',
        'softphysicalactivity']

df_temp = df_temp.loc[(df_temp['cvd']==0) & (df_temp['diabetes']==0) & (df_temp['depression']==0)]
df_temp = df_temp.loc[(df_temp['hardphysicalactivity']==0) & (df_temp['moderatephysicalactivity']==0) & (df_temp['softphysicalactivity']==0)]
df_big = df_big.drop(df_temp.index)
df_big.shape

(501238, 72)

In [28]:
print(df_big['depression'].value_counts())
print(df_big['diabetes'].value_counts())
print(df_big['cvd'].value_counts())

0    303739
1    197499
Name: depression, dtype: int64
0    472245
1     28993
Name: diabetes, dtype: int64
0    285008
1    216230
Name: cvd, dtype: int64


In [48]:
#dictionary specifying the data type i.c. categorical or int or cont for each column
feat_cat = {'sex': 'cat',
 'waistcircum': 'cont',
 'hipcircum': 'cont',
 'sleepduration': 'int',
 'napduringday': 'cat',
 'sleeplessness': 'cat',
 'daytimedozing': 'cat',
 'currenttobacco': 'cat',
 'cookedvegetable': 'int',
 'saladintake': 'int',
 'fruitintake': 'int',
 'driedfruit': 'int',
 'oilyfishintake': 'cat',
 'nonoilyfish': 'cat',
 'processedmeat': 'cat',
 'poultryintake': 'cat',
 'beefintake': 'cat',
 'lambintake': 'cat',
 'porkintake': 'cat',
 'cheeseintake': 'cat',
 'milkused': 'cat',
 'spreadtype': 'cat',
 'breadintake': 'int',
 'breadtype': 'cat',
 'cerealintake': 'int',
 'salttofood': 'cat',
 'teaintake': 'int',
 'waterintake': 'int',
 'majordietarychang': 'cat',
 'variationdiet': 'cat',
 'alcoholintake': 'cat',
 'adoptedaschild': 'cat',
 'freqdepressed': 'cat',
 'frequnenthusiasm2weeks': 'cat',
 'freqtensenness2weeks': 'cat',
 'freqtiredness2weeks': 'cat',
 'currentemployment': 'cat',
 'smoking': 'cat',
 'alcoholdrinkerstatus': 'cat',
 'ethnic': 'cat',
 'bmi': 'cont',
 'weight': 'cont',
 'bodyfatpercent': 'cont',
 'whfolebodyfatmass': 'cont',
 'wholebodyfatfreemass': 'cont',
 'summedMETminutes': 'cont',
 'qualification': 'cat',
 'agecompleted': 'int',
 'pasttobacco': 'cat',
 'cerealtype': 'cat',
 'coffetype': 'cat',
 'breastfedbaby': 'cat',
 'maternalsmoking': 'cat',
 'nonbutterspread': 'cat',    
 'depression': 'cat',
 'diabetes': 'cat',
 'cvd': 'cat',
 'someonetakedoctorchild': 'cat',
 'sexuallymolestedchild': 'cat',
 'feltlovedchild': 'cat',
 'physicallyabusedfamiliychild': 'cat',
 'felthatedfamiliychild': 'cat',
 'amountalcoholdrunk': 'cat',
 'everhadknownpersonalcoh': 'cat',
 'everbeeninjuredrinkalcohol': 'cat',
 'frequdrinkingalcohol': 'cat',
 'evertakencannabis': 'cat',
 'avoidedactsstresspasthmonth': 'cat',
 'repeatedisturbingpastmonth': 'cat',
 'feltupsetstressfulpastmonth': 'cat',
 'belittlementbypartneradult': 'cat',
 'beenconfidingrelationadult': 'cat',
 'physicalviolenceadult': 'cat',
 'sexualinterferenceadult': 'cat',
 'abletopaymortgageadult': 'cat',
 'beenseriousaccidentlifethrea': 'cat',
 'diagnosedlifethreateningillnes': 'cat',
 'victimphysicallyviolentcrime': 'cat',
 'witnessedsuddenviolentdeath': 'cat',
 'victimsexualassault': 'cat',
 'hardphysicalactivity': 'cat',
 'moderatephysicalactivity': 'cat',
 'softphysicalactivity': 'cat',
 'age': 'int',
 'townsenddeprivation': 'cont',
 'adoptedasachild': 'cat'
           }

len(feat_cat)

86

In [49]:
more_cols_drop = [ 'id','diastolicbloodpress',
 'systolicbloodpress',
 'everdepressed',
 'perioddepress',
 'numberdepressper',
 'everdesinteresedweek',
 'neuroticismscore',
 'Creactive']

df_big = df_big.drop(more_cols_drop, axis = 1)
df_big.shape

(501238, 63)

In [50]:
# subtract each value in column 'age' from 2012 to calculate age
df_big['age'] = 2012 - df_big['age']

In [51]:
df_big.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 501238 entries, 0 to 502480
Data columns (total 63 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   sex                       501238 non-null  float64
 1   waistcircum               499452 non-null  float64
 2   hipcircum                 499394 non-null  float64
 3   agecompleted              330142 non-null  float64
 4   sleepduration             497534 non-null  float64
 5   napduringday              499852 non-null  float64
 6   sleeplessness             500241 non-null  float64
 7   daytimedozing             498018 non-null  float64
 8   currenttobacco            500423 non-null  float64
 9   pasttobacco               459677 non-null  float64
 10  cookedvegetable           494170 non-null  float64
 11  saladintake               493912 non-null  float64
 12  fruitintake               498510 non-null  float64
 13  driedfruit                494800 non-null  f

## Managing missing values

### Fill using meadian, mode and mean

In [52]:
# use mode for categorical, mean for continuous and mode for integer type values

#--------for df_big------------------
fill_mbig = df_big.copy()
for col in fill_mbig.columns:
    if feat_cat[col] == 'cat' or feat_cat[col] == 'int' or feat_cat[col] == 'num':
        fill_mbig[col] = fill_mbig[col].fillna(fill_mbig[col].mode()[0])
    elif feat_cat[col] == 'cont':
        fill_mbig[col] = fill_mbig[col].fillna(fill_mbig[col].mean())
    else:
        print(col + " not in dict")

In [53]:
fill_mbig.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 501238 entries, 0 to 502480
Data columns (total 63 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   sex                       501238 non-null  float64
 1   waistcircum               501238 non-null  float64
 2   hipcircum                 501238 non-null  float64
 3   agecompleted              501238 non-null  float64
 4   sleepduration             501238 non-null  float64
 5   napduringday              501238 non-null  float64
 6   sleeplessness             501238 non-null  float64
 7   daytimedozing             501238 non-null  float64
 8   currenttobacco            501238 non-null  float64
 9   pasttobacco               501238 non-null  float64
 10  cookedvegetable           501238 non-null  float64
 11  saladintake               501238 non-null  float64
 12  fruitintake               501238 non-null  float64
 13  driedfruit                501238 non-null  f

In [54]:
# use mode for categorical, mean for continuous and mode for integer type values

#--------for df_small------------------
fill_msmall = df_small.copy()
for col in fill_msmall.columns:
    if feat_cat[col] == 'cat' or feat_cat[col] == 'int' or feat_cat[col] == 'num':
        fill_msmall[col] = fill_msmall[col].fillna(fill_msmall[col].mode()[0])
    elif feat_cat[col] == 'cont':
        fill_msmall[col] = fill_msmall[col].fillna(fill_msmall[col].mean())
    else:
        print(col + " not in dict")

In [55]:
fill_msmall.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 157287 entries, 0 to 502479
Data columns (total 26 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   amountalcoholdrunk              157287 non-null  float64
 1   everhadknownpersonalcoh         157287 non-null  float64
 2   everbeeninjuredrinkalcohol      157287 non-null  float64
 3   frequdrinkingalcohol            157287 non-null  float64
 4   evertakencannabis               157287 non-null  float64
 5   felthatedfamiliychild           157287 non-null  float64
 6   physicallyabusedfamiliychild    157287 non-null  float64
 7   feltlovedchild                  157287 non-null  float64
 8   sexuallymolestedchild           157287 non-null  float64
 9   someonetakedoctorchild          157287 non-null  float64
 10  avoidedactsstresspasthmonth     157287 non-null  float64
 11  repeatedisturbingpastmonth      157287 non-null  float64
 12  feltupsetstressf

### Fill using KNN imputer

In [38]:
from sklearn.impute import KNNImputer

fill_knn_ = df_cc.copy()
#change dtypes of columns, it's important for categorical features to have proper datatype
for col in fill_knn_.columns:
    if feat_cat[col] == 'cat':
        fill_knn_ = fill_knn_.astype({col:'category'})
    elif feat_cat[col] == 'int' :
        fill_knn_[col].replace({np.nan:45550}, inplace=True) #because astype doesnt convert nan
        fill_knn_ = fill_knn_.astype({col:'int64'})
        fill_knn_[col].replace({45550:np.nan}, inplace=True) #switch back to nan
    elif feat_cat[col] == 'cont' :
        fill_knn_ = fill_knn_.astype({col:'float64'})

fill_knn_.info()

# define imputer
imputer = KNNImputer()
# fit on the dataset
imputer.fit(fill_knn_)
# transform the dataset
fill_knn_ = imputer.transform(fill_knn_)
fill_knn = pd.DataFrame(fill_knn_, columns = df_cc.columns)
fill_knn.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 157072 entries, 0 to 502479
Data columns (total 86 columns):
 #   Column                          Non-Null Count   Dtype   
---  ------                          --------------   -----   
 0   sex                             157072 non-null  category
 1   waistcircum                     156842 non-null  float64 
 2   hipcircum                       156829 non-null  float64 
 3   agecompleted                    84759 non-null   float64 
 4   sleepduration                   156670 non-null  float64 
 5   napduringday                    157008 non-null  category
 6   sleeplessness                   156962 non-null  category
 7   daytimedozing                   156795 non-null  category
 8   currenttobacco                  156999 non-null  category
 9   pasttobacco                     149141 non-null  category
 10  cookedvegetable                 156253 non-null  float64 
 11  saladintake                     156168 non-null  float64 
 12  fr

In [56]:
#Mangaging categorical values
def manage_cat(df_ca):
    
    #In 'currenttobacco' column, switch 1:Yes, on most or all days and 2:Only occasionally to maintain an order
    df_ca['currenttobacco'].replace({1: 2, 2: 1}, inplace=True)

    # 'currentemployment' has following categories:
    # 1:In paid employment or self-employed
    # 2:Retired
    # 3:Looking after home and/or family
    # 4:Unable to work because of sickness or disability
    # 5:Unemployed
    # 6:Doing unpaid or voluntary work
    # 7:Full or part-time student
    # 8:None of the above

    #Some of them can be merged to simplify the columns as:
    # 0:Unemployed & Full or part-time student
    # 1:Looking after home and/or family & Doing unpaid or voluntary work & Unable to work because of sickness or disability & other
    # 2:Retired
    # 3:In paid employment or self-employed

    newvalsemp = {1:3, 3:1, 4:1, 5:0, 6:1, 7:0, 8:1}
    print("df_ca['currentemployment'] unique: ", df_ca['currentemployment'].unique())
    df_ca['currentemployment'] = df_ca['currentemployment'].replace(newvalsemp)

    # 'ethnic' has many categories but they can be merged together to obtain fewer categories (using only top level categories in UKBB for this feature) 
    # New categories could be like this:
    # 1 (White): 1001 & 1002 & 1003
    # 2 (Mixed): 2001 & 2002 & 2003 & 2004
    # 3 (Asian or Asian British): 3001 & 3002 & 3003 & 3004 & 5
    # 4 (Black or Black British): 4001 & 4002 & 4003
    # 5 (other): 6

    
    newvalseth = {1001:1, 1002:1, 1003:1, 2001:2, 2002:2, 2003:2, 2004:2, 3001:3, 3002:3, 3003:3, 3004:3, 
                  5:3, 4001:4, 4002:4, 4003:4, 6:5}
    df_ca['ethnic'] = df_ca['ethnic'].replace(newvalseth)

    # 'majordietarychang' has following categories:
    # 0:No
    # 1:Yes, on most or all days
    # 2:Only occasionally

    # 1 and 2 can be merged for simplification
    df_ca['majordietarychang'].replace({2: 1}, inplace=True)
    
    # creating instance of one-hot-encoder
    enc = OneHotEncoder(handle_unknown='ignore')
    # passing currentemployment column (label encoded values of currentemployment)
    enc_emp = pd.DataFrame(enc.fit_transform(df_ca[['currentemployment']]).toarray())
    
    #update new column names
    employment_status = ['Unemployed', 'not_working', 'retired', 'employed']
    enc_emp   = enc_emp.set_axis(employment_status, axis=1, inplace=False)
    enc_emp.reset_index(drop=True, inplace=True)# reset the index of the dataframe
    # passing ethnic column (label encoded values of ethnic)
    enc_eth = pd.DataFrame(enc.fit_transform(df_ca[['ethnic']]).toarray())
    print(enc_emp.isna().sum())
    print("enc_emp.shape: ", enc_emp.shape)
    
    #update new column names
    ethnicities = ['White', 'Mixed', 'AsianorAsianBritish', 'BlackorBlackBritish', 'otherEth']
    enc_eth   = enc_eth.set_axis(ethnicities, axis=1, inplace=False)
    enc_eth.reset_index(drop=True, inplace=True)# reset the index of the dataframe
    print(enc_eth.isna().sum())
    print("enc_eth.shape: ", enc_eth.shape)
    df_ca.reset_index(drop=True, inplace=True)# reset the index of the dataframe
    print(df_ca.isna().sum())
    print("df_ca.shape: ", df_ca.shape)
    # merge new dataframes ie enc_emp and enc_eth with main df on key values
    df_ca = df_ca.join(enc_emp)
    df_ca = df_ca.join(enc_eth)

    #Remove the columns 'currentemployment' and 'ethnic' from the main df
    df_ca = df_ca.drop(['currentemployment', 'ethnic'], axis = 1)
    
    print(df_ca.isna().sum())
    print("df_ca.shape: ", df_ca.shape)
    
    return df_ca

In [57]:
fill_mbig = manage_cat(fill_mbig)
fill_mbig.to_csv('fill_mbig.csv', index=False)

df_ca['currentemployment'] unique:  [1. 2. 5. 8. 4. 3. 7. 6.]
Unemployed     0
not_working    0
retired        0
employed       0
dtype: int64
enc_emp.shape:  (501238, 4)
White                  0
Mixed                  0
AsianorAsianBritish    0
BlackorBlackBritish    0
otherEth               0
dtype: int64
enc_eth.shape:  (501238, 5)
sex                 0
waistcircum         0
hipcircum           0
agecompleted        0
sleepduration       0
                   ..
summedMETminutes    0
qualification       0
depression          0
diabetes            0
cvd                 0
Length: 63, dtype: int64
df_ca.shape:  (501238, 63)
sex                    0
waistcircum            0
hipcircum              0
agecompleted           0
sleepduration          0
                      ..
White                  0
Mixed                  0
AsianorAsianBritish    0
BlackorBlackBritish    0
otherEth               0
Length: 70, dtype: int64
df_ca.shape:  (501238, 70)


In [58]:
fill_msmall.to_csv('fill_msmall.csv', index=False)

In [74]:
fill_knn1 = manage_cat(fill_knn1)
fill_knn1.to_csv('fill_knn1.csv', index=False)

df_ca['currentemployment'] unique:  [3.  2.  1.  0.  1.2 2.4 1.4 1.6 2.8 1.8 2.6 3.2 4.2 2.2 3.6]


ValueError: Length mismatch: Expected axis has 15 elements, new values have 4 elements